In [ ]:
%%capture
!git clone https://github.com/DuyMinhLe13/Double-Agent-Tetris
!pip install pygame
!git clone https://github.com/DuyMinhLe13/notebook-video-writer
import os
import shutil
shutil.move('/content/Double-Agent-Tetris/TetrisBattle', '/content')
shutil.move('/content/Double-Agent-Tetris/CustomAgent.py', '/content')
shutil.move('notebook-video-writer/notebook_video_writer', '/content')

In [ ]:
from TetrisBattle.envs.tetris_env import TetrisDoubleEnv, TetrisSingleEnv
from TetrisBattle.envs.tetris_env import TetrisSingleInterface
import numpy as np
from notebook_video_writer import VideoWriter

  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel



In [ ]:
import numpy as np
import copy

GRID_WIDTH = 10
GRID_DEPTH = 20

BLOCK_LENGTH = 4
BLOCK_WIDTH = 4

ipieces = [[[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]],
          [[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0]],
          [[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]],
          [[0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0]]]
opieces = [[[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 2, 2, 0], [0, 2, 2, 0], [0, 0, 0, 0]]]
jpieces = [[[0, 3, 3, 0], [0, 0, 3, 0], [0, 0, 3, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 3, 3, 3], [0, 3, 0, 0], [0, 0, 0, 0]],
          [[0, 0, 3, 0], [0, 0, 3, 0], [0, 0, 3, 3], [0, 0, 0, 0]],
          [[0, 0, 0, 3], [0, 3, 3, 3], [0, 0, 0, 0], [0, 0, 0, 0]]]
lpieces = [[[0, 0, 4, 0], [0, 0, 4, 0], [0, 4, 4, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 4, 4, 4], [0, 0, 0, 4], [0, 0, 0, 0]],
          [[0, 0, 4, 4], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 0, 0]],
          [[0, 4, 0, 0], [0, 4, 4, 4], [0, 0, 0, 0], [0, 0, 0, 0]]]
zpieces = [[[0, 5, 0, 0], [0, 5, 5, 0], [0, 0, 5, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 5, 5, 0], [5, 5, 0, 0], [0, 0, 0, 0]],
          [[0, 5, 0, 0], [0, 5, 5, 0], [0, 0, 5, 0], [0, 0, 0, 0]],
          [[0, 0, 5, 5], [0, 5, 5, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
spieces = [[[0, 0, 6, 0], [0, 6, 6, 0], [0, 6, 0, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 6, 6, 0], [0, 0, 6, 6], [0, 0, 0, 0]],
          [[0, 0, 6, 0], [0, 6, 6, 0], [0, 6, 0, 0], [0, 0, 0, 0]],
          [[6, 6, 0, 0], [0, 6, 6, 0], [0, 0, 0, 0], [0, 0, 0, 0]]]
tpieces = [[[0, 0, 7, 0], [0, 7, 7, 0], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 0, 0], [0, 7, 7, 7], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 7, 0], [0, 0, 7, 7], [0, 0, 7, 0], [0, 0, 0, 0]],
          [[0, 0, 7, 0], [0, 7, 7, 7], [0, 0, 0, 0], [0, 0, 0, 0]]]
lspieces = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] #this is the lines sent piece aka garbage lines

PIECES_DICT = {
    'I': ipieces, 'O': opieces, 'J': jpieces,
    'L': lpieces, 'Z': zpieces, 'S': spieces,
    'T': tpieces, 'G': lspieces
}

PIECE_NUM2TYPE = {1: 'I', 2: 'O', 3: 'J', 4: 'L', 5: 'Z', 6: 'S', 7: 'T', 8: 'G'}
PIECE_TYPE2NUM = {val: key for key, val in PIECE_NUM2TYPE.items()}
POSSIBLE_KEYS = ['I', 'O', 'J', 'L', 'Z', 'S', 'T']

def heightForColumn(grid, column):
    height, width = grid.shape
    for i in range(0, height):
        if grid[i][column] != 0:
            return height-i
    return 0

def heights(grid):
    result = []
    height, width = grid.shape
    for i in range(0, width):
        result.append(heightForColumn(grid, i))
    return result

def numberOfHoleInColumn(grid, column):
    result = 0
    maxHeight = heightForColumn(column)
    for height, line in enumerate(reversed(grid)):
        if height > maxHeight: break
        if line[column] == 0 and height < maxHeight:
            result+=1
    return result

def numberOfHoleInRow(grid, line):
    result = 0
    height, width = grid.shape
    for index, value in enumerate(grid[height-1-line]):
        if value == 0 and heightForColumn(index) > line:
            result += 1
    return result
def heuristics(grid):
    columns_heights = heights(grid)
    return columns_heights + [aggregateHeight(columns_heights)] + numberOfHoles(grid, columns_heights) + bumpinesses(columns_heights) + [completLine(grid), maxPitDepth(columns_heights), maxHeightColumns(columns_heights), minHeightColumns(columns_heights)]

def heuristics_for_oppo(grid):
    columns_heights = heights(grid)
    return columns_heights + [aggregateHeight(columns_heights)] + numberOfHoles(grid, columns_heights) + bumpinesses(columns_heights) + [maxPitDepth(columns_heights), maxHeightColumns(columns_heights), minHeightColumns(columns_heights)]

def aggregateHeight(heights):
    result = sum(heights)
    return result

def completLine(grid):
    result = 0
    height, width = grid.shape
    for i in range (0, height) :
        if 0 not in grid[i]:
            result+=1
    return result

def bumpinesses(heights):
    result = []
    for i in range(0, len(heights)-1):
        result.append(abs(heights[i]-heights[i+1]))
    return result

def numberOfHoles(grid, heights):
    results = []
    height, width = grid.shape
    for j in range(0, width) :
        result = 0
        for i in range (0, height) :
            if grid[i][j] == 0 and height-i < heights[j]:
                result+=1
        results.append(result)
    return results

def maxHeightColumns(heights):
    return max(heights)

def minHeightColumns(heights):
    return min(heights)

def maxPitDepth(heights):
    return max(heights)-min(heights)

In [ ]:
def get_feasible(block):
    feasibles = []

    b = block

    for x in range(BLOCK_WIDTH):
        for y in range(BLOCK_LENGTH):
            if b[x][y] > 0:
                feasibles.append([x, y])

    return feasibles

def collide(grid, block, px, py):
    feasibles = get_feasible(block)

    for pos in feasibles:
        # print(px + pos[0], py + pos[1])
        if px + pos[0] > GRID_WIDTH - 1:   # right
            return True

        if px + pos[0] < 0:   # left
            return True

        if py + pos[1] > len(grid[0]) - 1:  # down
            return True

        if py + pos[1] < 0:   # up
            continue

        if grid[px + pos[0]][py + pos[1]] > 0:
            # print(px, py)
            # print(px + pos[0], py + pos[1])
            # print("Touch")
            return True

    return False

def collideDown(grid, block, px, py):
    return collide(grid, block, px, py + 1)

def hardDrop(grid, block, px, py):
    y = 0
    x = 0
    if collideDown(grid, block, px, py) == False:
        x = 1
    if x == 1:
        while True:
            py += 1
            y += 1
            if collideDown(grid, block, px, py) == True:
                break

    return y

def get_block_list_id(one_hot):
  for i in range(len(one_hot)):
    if one_hot[i] == 1:
      return i + 1
  return 0

def get_grid(px, block, state):
    return_grids = np.zeros(shape=(10, 20), dtype=np.float32)

    grid = copy.deepcopy(state[:, :10].reshape(20, 10).transpose())
    grid[grid == 0.7] = 0.0
    grid[grid == 0.3] = 0.0

    block, px, py = block, px, -2

    add_y = hardDrop(grid, block, px, py)

    for x in range(0,4):
        for y in range(0,4):
            if block[x][y] > 0:
                # draw ghost grid
                if -1 < px + x < 10 and -1 < py + y + add_y < 20:
                    grid[px + x][py + y + add_y] = 0.3

                if -1 < px + x < 10 and -1 < py + y < 20:
                    grid[px + x][py + y] = 0.7
    return grid.transpose()

def cnt_cleared_lines(state):
    state = state[:, :10].reshape(20, 10)
    state[state == 0.3] = 1.0
    state[state != 1.0] = 0.0
    cnt_full_lines = 0
    for line in state:
      check = True
      for cell in line:
        if cell == 0:
          check = False
          break
      if check == True:
        cnt_full_lines += 1

    return cnt_full_lines

def lines_for_combo(n_combos):
    if n_combos == -1:
      return -1
    elif n_combos == 0:
      return 0
    elif n_combos == 1 or n_combos == 2:
      return 1
    elif n_combos == 3 or n_combos == 4:
      return 2
    elif n_combos == 5 or n_combos == 6:
      return 3
    else:
      return 4


class Agent:
    def __init__(self, turn):
        # dir_path = os.path.dirname(os.path.realpath(__file__))
        # weight_file_path = os.path.join(dir_path, turn, 'weight')

        self.moves = []
        self.weights = []

    def best_action(self, state):
        n_block_list_id = get_block_list_id(state[6, 10:17].reshape(7))
        h_block_list_id = get_block_list_id(state[0, 10:17].reshape(7))
        f_block_list_id = get_block_list_id(state[1, 10:17].reshape(7))

        bestRotation = None
        bestOffset = None
        bestScore = None
        bestSwap = None

        possible_blocks = []

        if h_block_list_id == 0:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[f_block_list_id]]]
        else:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]]]

        bestScore_ = [0, 0]
        bestOffset_ = [0, 0]
        bestRotation_ = [0, 0]

        my_combo = copy.deepcopy(state[7, 11]).reshape(1)*10
        block_infos = copy.deepcopy(state[0:7, 10:17]).reshape(7,7).flatten()
        lines_about_to_get = copy.deepcopy(state[7, 13]).reshape(1)*20

        for i in range(len(possible_blocks)):
          for rotation in range(0, 4):
              cnt_check = np.count_nonzero(np.array(possible_blocks[i][rotation]).flatten() != 0)

              for offset in range(-3, 11):
                changed_board = get_grid(offset, possible_blocks[i][rotation], state)

                if np.count_nonzero(np.array(changed_board).flatten() == 0.3) == cnt_check:
                    changed_board[changed_board == 0.3] = 1.0
                    changed_board[changed_board != 1.0] = 0.0

                    heuristic = np.concatenate((heuristics(changed_board),my_combo,lines_about_to_get,block_infos))

                    score = sum([a*b for a,b in zip(heuristic, self.weights)])

                    if bestScore_[i] == 0 or score > bestScore_[i]:
                        bestScore_[i] = score
                        bestOffset_[i] = offset
                        bestRotation_[i] = rotation


        if bestScore_[0] > bestScore_[1]:
          bestRotation = bestRotation_[0]
          bestOffset = bestOffset_[0]
          bestScore = bestScore_[0]
          bestSwap = 0
        else:
          bestRotation = bestRotation_[1]
          bestOffset = bestOffset_[1]
          bestScore = bestScore_[1]
          bestSwap = 1

        return bestOffset, bestRotation, bestSwap, bestScore

    def choose_action(self, state):
        if(len(self.moves) > 0):
          move =self.moves[0]
          self.moves.pop(0)
          return move

        offset, rotation, swap, _ = Agent.best_action(self, state)

        if swap == 1:
          self.moves.append(1)

        if offset != None:
          offset = offset - 4
          if rotation == 3:
            self.moves.append(4)
          else:
            for _ in range(0, rotation):
                self.moves.append(3)

          for _ in range(0, abs(offset)):
              if offset > 0:
                  self.moves.append(5)
              else:
                  self.moves.append(6)

          self.moves.append(2)

        move =self.moves[0]
        self.moves.pop(0)
        return move

class Agent_op:
    def __init__(self, turn):
        # dir_path = os.path.dirname(os.path.realpath(__file__))
        # weight_file_path = os.path.join(dir_path, turn, 'weight')

        self.moves = []
        self.weights = [-0.3863239645030658, 0.04624820118746831, -1.1356125375827606, -0.3945298370949794, -0.1821421606665584, -0.5968781507045787, 0.2781728756286588, 0.2477254430012728, -0.3775786813407028, 0.2532399807483993, -2.0071959356079527, 0.006788952440678342, -0.5010904756423058, -0.2425072518648133, 0.04178465070991613, -1.6386822022346967, -1.8300055988357031, 0.8297504893120988, -1.519829456757352, -1.265196087485425, -1.8424907993494721, 0.13690198742896076, 0.5264956307205383, -1.0255406723974851, -0.577119174439163, -1.077941699626955, -1.8209966061797491, -1.274816828927968, -1.5789242564661508, -0.0361383041831671, 2.7767906583769797, -1.9301510798480186, -0.9323251918785673, 0.1981848462704232]

    def best_action(self, state):
        n_block_list_id = get_block_list_id(state[6, 10:17].reshape(7))
        h_block_list_id = get_block_list_id(state[0, 10:17].reshape(7))
        f_block_list_id = get_block_list_id(state[1, 10:17].reshape(7))

        bestRotation = None
        bestOffset = None
        bestScore = None
        #bestSwap = 1 thì vứt block hiện tại vào hold và lấy block đầu tiên ở tương lai
        #bestSwap = 0 thì sử dụng block hiện tại
        #bestSwap = 1 thì hoán đổi block hiện tại với block trong hold
        bestSwap = None

        possible_blocks = []

        if h_block_list_id == 0:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[f_block_list_id]]]
        else:
          possible_blocks = [PIECES_DICT[PIECE_NUM2TYPE[n_block_list_id]], PIECES_DICT[PIECE_NUM2TYPE[h_block_list_id]]]

        bestScore_ = [0, 0]
        bestOffset_ = [0, 0]
        bestRotation_ = [0, 0]

        for i in range(len(possible_blocks)):
          for rotation in range(0, 4):
              cnt_check = np.count_nonzero(np.array(possible_blocks[i][rotation]).flatten() != 0)

              for offset in range(-3, 11):
                changed_board = get_grid(offset, possible_blocks[i][rotation], state)

                if np.count_nonzero(np.array(changed_board).flatten() == 0.3) == cnt_check:
                    changed_board[changed_board == 0.3] = 1.0
                    changed_board[changed_board != 1.0] = 0.0

                    heuristic = heuristics(changed_board)

                    score = sum([a*b for a,b in zip(heuristic, self.weights)])

                    if bestScore_[i] == 0 or score > bestScore_[i]:
                        bestScore_[i] = score
                        bestOffset_[i] = offset
                        bestRotation_[i] = rotation


        if bestScore_[0] > bestScore_[1]:
          bestRotation = bestRotation_[0]
          bestOffset = bestOffset_[0]
          bestScore = bestScore_[0]
          bestSwap = 0
        else:
          bestRotation = bestRotation_[1]
          bestOffset = bestOffset_[1]
          bestScore = bestScore_[1]
          bestSwap = 1

        return bestOffset, bestRotation, bestSwap, bestScore

    def choose_action(self, state):
        if(len(self.moves) > 0):
          move =self.moves[0]
          self.moves.pop(0)
          return move

        offset, rotation, swap, _ = Agent_op.best_action(self, state)

        if swap == 1:
          self.moves.append(1)

        if offset != None:
          offset = offset - 4
          if rotation == 3:
            self.moves.append(4)
          else:
            for _ in range(0, rotation):
                self.moves.append(3)

          for _ in range(0, abs(offset)):
              if offset > 0:
                  self.moves.append(5)
              else:
                  self.moves.append(6)

          self.moves.append(2)

        move =self.moves[0]
        self.moves.pop(0)
        return move

In [ ]:
import random
import math
import statistics
import time
from operator import itemgetter
from sys import stdout
from functools import reduce

def individualFromDistribution(average, std):
    result = []
    for i in range(0, size):
        result.append(random.normalvariate(average[i], std[i]))
    return result

def generationFromDistribution(number, size, average, std):
    results = []
    for i in range(0, number):
        tmp = individualFromDistribution(average, std)
        results.append(tmp)
    return results

def selectBestIndividuals(population, scores, number):
    bests = [individual for _, individual in reversed(sorted(zip(scores, population)))][0:number]
    return bests

def computeAverage(population):
    result = list(reduce(lambda i1, i2: [a+b for a,b in zip(i1, i2)], population))
    result = list(map(lambda x: x/len(population), result))
    return result

def computeStandardDeviation(population):
    average = computeAverage(population)
    result = [[] for _ in range(0, len(population[0]))]
    for individual in population:
        for index, weight in enumerate(individual):
            result[index].append(weight)
    result = list(map(lambda weights: statistics.stdev(weights), result))
    return result

In [ ]:
from numpy.lib.function_base import average
from tqdm import tqdm

survivors_rate = 0.2
pieceLimit = 1000
number = 50
batch = 100
size = 85
mutation_rate = 0.05

env = TetrisDoubleEnv(mode="rgb_array")
# agent = Agent(0)
agent_list = [Agent(0), Agent_op(1)]

generation = []
fitness_values = []
average = [0 for i in range(size)]
std = [1 for i in range(size)]

# for indivual in generationFromDistribution(number - len(generation), size, average, std):
#   generation.append(indivual)


# for i in range(0, batch):
#   fitness_values = []
#   for individual in tqdm(generation):
#     done = False
#     state = env.reset()
#     # env.game_interface.time = 1000000 * 3 / 16
#     # agent.weights = individual
#     agent_list[0].weights = individual

#     lines_cleared = 0
#     n_block_fallen = 0
#     infos = None
#     while not done:
#         img = env.render(mode='rgb_array')
#         action = agent_list[env.game_interface.getCurrentPlayerID()].choose_action(state)
#         # action = agent.choose_action(state)
#         if env.game_interface.getCurrentPlayerID() == 0:
#           if action == 2:
#             if int(state[7, 13].reshape(1)[0] * 20) == 0:
#               lines_cleared += ((cnt_cleared_lines(state) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)
#             else:
#               if (cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20) < 0:
#                 lines_cleared += (-10*((cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)
#               else:
#                 lines_cleared += (((cnt_cleared_lines(state) - 1) - int(state[7, 13].reshape(1)[0] * 20))**2 + lines_for_combo(int(state[7, 11].reshape(1)[0] * 10))**2)

#             n_block_fallen += 1
#         state, reward, done, infos = env.step(action)
#         env.take_turns()

#     fitness_score = lines_cleared * 1000 + n_block_fallen
#     if infos['winner'] == 1:
#       fitness_score -= 500000 * state[19, 16].reshape(1)[0]
#     fitness_values.append(fitness_score)

#   rev_sort_fitness = fitness_values
#   rev_sort_fitness = [score for score, individual in reversed(sorted(zip(fitness_values, generation)))]
#   rev_sort_gene = [individual for score, individual in reversed(sorted(zip(fitness_values, generation)))]

#   print("Generation", i + 1)
#   print("Score: ", rev_sort_fitness)
#   print("Weights: ", rev_sort_gene)
#   new_generation = selectBestIndividuals(generation, fitness_values, int(number * survivors_rate))

#   average = computeAverage(new_generation)
#   std = list(map(lambda std: std, computeStandardDeviation(new_generation)))

#   for indivual in generationFromDistribution(number - len(new_generation), size, average, std):
#     new_generation.append(indivual)

#   generation = new_generation


In [ ]:
# print(generation)
generation = [
    [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
     [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
      [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
       [-3.29845866046991, 2.9634312417576876, 2.9464536194246005, 6.368070584380806],
        [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
         [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
          [-8.061238470111324, 7.166357355281924, 8.45090956744317, 6.502608878463899],
           [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
            [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
             [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
              [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
               [-8.061238470111324, 7.166357355281924, 8.45090956744317, 6.502608878463899],
                [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
                 [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
                  [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
                   [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
                    [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823],
                     [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823], [3.395075499415878, -8.30888596024571, 3.3864979356650355, -6.232257534041823]]

# best_individual = [individual for _, individual in reversed(sorted(zip(fitness_values, generation)))][0]

best_individual = [-0.3863239645030658, 0.04624820118746831, -1.1356125375827606, -0.3945298370949794, -0.1821421606665584, -0.5968781507045787, 0.2781728756286588, 0.2477254430012728, -0.3775786813407028, 0.2532399807483993, -2.0071959356079527, 0.006788952440678342, -0.5010904756423058, -0.2425072518648133, 0.04178465070991613, -1.6386822022346967, -1.8300055988357031, 0.8297504893120988, -1.519829456757352, -1.265196087485425, -1.8424907993494721, 0.13690198742896076, 0.5264956307205383, -1.0255406723974851, -0.577119174439163, -1.077941699626955, -1.8209966061797491, -1.274816828927968, -1.5789242564661508, -0.0361383041831671, 2.7767906583769797, -1.9301510798480186, -0.9323251918785673, 0.1981848462704232]

env = TetrisSingleEnv(mode="rgb_array")
agent = Agent(0)
agent.weights = generation[0]

done = False
state = env.reset()

with VideoWriter(fps=60) as vw:
  while not done:
      vw.add(env.render(mode='rgb_array')) # img is rgb array, you need to render this or can check my colab notebook in readme file
      action = agent.choose_action(state)
      state, reward, done, _ = env.step(action)

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

